In [1]:
import requests
import json
import pandas as pd

# Initialize Variables and authenticate

In [2]:
# initialize variables
auth_url = "https://api.hatebase.org/4-2/authenticate"
vocab_url = "https://api.hatebase.org/4-2/get_vocabulary"
lang = "eng"
resp_format = "json"
token = ""
pages = 0
total_entries = 0

In [3]:
# initialize authentication payload
auth_payload = "api_key=mmKPRoEEPSCHTI34598dhtP"
headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache"
    }

In [4]:
# authenticate
auth_resp = requests.request("POST", auth_url, data=auth_payload, headers=headers)

In [5]:
# get the session token
token = auth_resp.json()["result"]["token"]

In [ ]:
# quick check if it worked
print(auth_resp.text, token)

# Initialize Vocabulary payload and build df

In [7]:
# initialize vocabulary payload
# first without any given page-number
vocab_payload = "token=" + token + "&format=" + resp_format + "&language=" + lang
voc_resp = requests.request("POST", vocab_url, data=vocab_payload, headers=headers)
voc_json = voc_resp.json()

If you have a look at the received voc_json (i.e. print it), it should look somewhat similar to this:
```
{
    "datetime": "2019-03-11 08:04:08",
    "token": "kiooZAEGnEdxjXhPtxNqVVNLrNfMPoXA",
    "format": "json",
    "language": "eng",
    "number_of_results": 1384,
    "number_of_pages": 14,
    "page": 1,
    "result": [
        {
            "vocabulary_id": "Id0fThatVoc4b",
            "term": "some-term",
```
            
etc.

In [9]:
# how many pages in total? and how many results= entries in the vocab?
pages = voc_json["number_of_pages"]
results = voc_json["number_of_results"]

In [10]:
# check pages & results
pages, results

(14, 1384)

In [11]:
# create vocabulary df from first resultset
df_voc = pd.DataFrame(voc_json["result"])

In [12]:
# now get results of all remaining pages and append to df_voc
for page in range(2,pages+1):
    vocab_payload = "token=" + token + "&format=json&language=" + lang + "&page=" + str(page)
    voc_resp = requests.request("POST", vocab_url, data=vocab_payload, headers=headers)
    voc_json = voc_resp.json()
    df_voc = df_voc.append(voc_json["result"])

# reset df_voc index
df_voc.reset_index(drop=True, inplace=True)

In [13]:
df_voc.shape

(1384, 23)

## save hate vocabulary to file

In [19]:
df_voc.to_csv("hatebase_vocab.csv")